In [1]:
import torch
import torch.nn as nn 
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import os
from PIL import Image
import dlib
import PinkBlack.trainer

In [2]:
IMAGE_FOLDER = "datasets/ImageFolder/"

In [3]:
df_train = pd.read_csv("datasets/train.csv")
df_test = pd.read_csv("datasets/test.csv")
print(df_train.shape)
print(df_test.shape)

(3200, 2)
(800, 2)


In [108]:
net = torchvision.models.squeezenet1_1(pretrained=True)
net.num_classes = 1
net.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512,1, kernel_size=(1,1), stride=(1,1)),
    nn.ReLU(inplace=True),
    nn.AvgPool2d(kernel_size=13, stride=1, padding=0)
)
net.cuda()
net.train()
'.'

'.'

In [5]:
detector = dlib.get_frontal_face_detector()

In [34]:

class BeautySet(torch.utils.data.Dataset):
    def __init__(self, df_data, mode='train'):
        self.samples = df_data.filename
        self.scores = df_data.score
        if mode=='train':
            self.transform = transforms.Compose([transforms.RandomRotation(10),
                                            transforms.RandomResizedCrop((224), scale=(0.7, 1.0)),
                                             
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor()])
        else:
            self.transform = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.ToTensor()])
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        img = Image.open(IMAGE_FOLDER + self.samples[index]).convert("RGB")
        dets = detector(np.array(img))
        if len(dets) == 0:
            dets = [0, 0, img.size[1], img.size[0]]
        else:
            dets = dets[0]
            w, h = dets.right() - dets.left(), dets.bottom() - dets.top()
            
            dets = [max(0, dets.left() - w // 5),
                    max(0, dets.top() - h // 5),
                    min(img.width, dets.right() + w // 5),
                    min(img.height, dets.bottom() + h // 5)]
            img = img.crop(dets)
        
        return self.transform(img), torch.tensor(self.scores[index])
        
        
        
        

In [106]:
train_sets = BeautySet(df_train, 'test')
test_sets = BeautySet(df_test, 'test')
train_loader = torch.utils.data.DataLoader(train_sets, batch_size=32, shuffle=True, pin_memory=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_sets, batch_size=32, shuffle=False, pin_memory=True, num_workers=0)


In [109]:
def metric(pred, target):
    loss = nn.MSELoss()
    with torch.no_grad():
        return loss(pred, target)

criterion = nn.MSELoss()
    

In [110]:
trainer = PinkBlack.trainer.Trainer(net=net,
                                    train_dataloader=train_loader,
                                   val_dataloader=test_loader,
                                   criterion=criterion,
                                   metric=metric,
                                   )

In [111]:
trainer.train(5)

|----------------|----------------|----------------|----------------|----------------|----------------|
|     epoch      |   train_loss   |  train_metric  |    val loss    |   val metric   |      time      |
|----------------|----------------|----------------|----------------|----------------|----------------|


|     1-best     |    1.467821    |    1.467821    |    0.536956    |    0.536956    |   88.750288    |
|----------------|----------------|----------------|----------------|----------------|----------------|


|       2        |    0.561579    |    0.561579    |    0.717687    |    0.717687    |   88.306579    |
|----------------|----------------|----------------|----------------|----------------|----------------|


|       3        |    0.549337    |    0.549337    |    0.666719    |    0.666719    |   87.631858    |
|----------------|----------------|----------------|----------------|----------------|----------------|


|       4        |    0.548195    |    0.548195    |    0.640637    |    0.640637    |   89.378845    |
|----------------|----------------|----------------|----------------|----------------|----------------|


|     5-best     |    0.516269    |    0.516269    |    0.523467    |    0.523467    |   90.295555    |
|----------------|----------------|----------------|----------------|----------------|----------------|


In [112]:
idx = 0

In [144]:
idx += 1
print(net(train_sets[idx][0].unsqueeze(0).cuda()))
train_sets[idx][1]

tensor([[3.0528]], device='cuda:0', grad_fn=<ViewBackward>)


tensor(2.8167)

In [147]:
torch.onnx.export(net, train_sets[0][0].unsqueeze(0).cuda(), "onnx_test.onnx", verbose=True)

C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\onnx\symbolic.py:131: UserWarning: ONNX export failed on max_pool2d_with_indices because ceil_mode not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


graph(%0 : Float(1, 3, 224, 224)
      %1 : Float(64, 3, 3, 3)
      %2 : Float(64)
      %3 : Float(16, 64, 1, 1)
      %4 : Float(16)
      %5 : Float(64, 16, 1, 1)
      %6 : Float(64)
      %7 : Float(64, 16, 3, 3)
      %8 : Float(64)
      %9 : Float(16, 128, 1, 1)
      %10 : Float(16)
      %11 : Float(64, 16, 1, 1)
      %12 : Float(64)
      %13 : Float(64, 16, 3, 3)
      %14 : Float(64)
      %15 : Float(32, 128, 1, 1)
      %16 : Float(32)
      %17 : Float(128, 32, 1, 1)
      %18 : Float(128)
      %19 : Float(128, 32, 3, 3)
      %20 : Float(128)
      %21 : Float(32, 256, 1, 1)
      %22 : Float(32)
      %23 : Float(128, 32, 1, 1)
      %24 : Float(128)
      %25 : Float(128, 32, 3, 3)
      %26 : Float(128)
      %27 : Float(48, 256, 1, 1)
      %28 : Float(48)
      %29 : Float(192, 48, 1, 1)
      %30 : Float(192)
      %31 : Float(192, 48, 3, 3)
      %32 : Float(192)
      %33 : Float(48, 384, 1, 1)
      %34 : Float(48)
      %35 : Float(192, 48, 1, 1)
      %36

RuntimeError: ONNX export failed: Couldn't export operator aten::max_pool2d_with_indices

Defined at:
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\nn\functional.py(417): max_pool2d_with_indices
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\nn\functional.py(425): _max_pool2d
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\_jit_internal.py(132): fn
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\nn\modules\pooling.py(148): forward
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\nn\modules\module.py(477): _slow_forward
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\nn\modules\module.py(487): __call__
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\nn\modules\container.py(92): forward
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\nn\modules\module.py(477): _slow_forward
C:\Users\sweetcocoa\Anaconda3\envs\pt\lib\site-packages\torch\nn\modules\module.py(487): __call__
C:\Us